## Table of content

In [ ]:
!pip install lxml
!pip install requests
!pip install beautifulsoup4
!pip install pymorphy2

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from google.colab import files

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import pymorphy2

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import gensim
import gensim.downloader
from gensim.models import Word2Vec,KeyedVectors

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Спарсим стихотворения с сайта: https://rustih.ru/

### Создание скрипта скрапинга

In [ ]:
url = 'https://rustih.ru/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

### Парсинг html разметки, спарсили талицу с сылками

In [ ]:
tables = soup.find_all('table')

# Посмотрим, какие стихи есть на сайте
for table in tables:
    print(table.text)

### Создадим датафрейм

In [ ]:
df = pd.DataFrame(columns=['Название', 'Ссылка', 'Текст'])

### Перейдем по ссылке на каждого автора, а затем на каждое стихотворение и добавим данные в датафрейм

In [ ]:
for table in tables:
    # Получаем ссылки на страницы из тегов <a>
    links = table.find_all('a')

    for link in links:
        # Получаем ссылку на страницу, где у нас есть список стихов определенного автора
        page_url_author = link.get('href')

        # Получаем содержимое страницы по ссылке
        page_response_author = requests.get(page_url_author)
        page_soup_author = BeautifulSoup(page_response_author.text, 'lxml')

        poems_links = page_soup_author.select('div.entry-title a')

        for poem_link in poems_links:
            poem_url = poem_link.get('href')

            poem_response = requests.get(poem_url)
            poem_soup = BeautifulSoup(poem_response.text, 'lxml')

            # Получаем название стихотворения и проверяем на пустоту
            title_element = poem_soup.select_one('h1.entry-title')
            title = title_element.text if title_element is not None else ''

            # Получаем текст стихотворения
            poem_lines = poem_soup.select('div.entry-content p')
            poem_text = ''

            for line in poem_lines:
                # Проверяем, что мы не вышли за пределы содержания стихотворения
                prev_sibling = line.find_previous_sibling()
                if prev_sibling is not None and prev_sibling.name == 'h2':
                    break
                poem_text += line.text + '\n'

            df = df.append({'Название': title, 'Ссылка': poem_url, 'Текст': poem_text}, ignore_index=True)

In [ ]:
df.sample(n=10)
df.shape[0]

### Скачаем датафрейм

In [ ]:
df.to_csv("table_poems.csv", index=False)
files.download("table_poems.csv")

## Предобработаем данные

### Импортируем файл

In [ ]:
data = pd.read_csv('/content/table_poems.csv', delimiter=',', quotechar='"', escapechar='\\')
data.head()

,Название,Ссылка,Текст
0,Иван Крылов — Свинья под дубом (Басня): Стих,https://rustih.ru/ivan-krylov-svinya-pod-dubom...,Свинья под Дубом вековым\nНаелась желудей досы...
1,Иван Крылов — Мартышка и очки (Басня): Стих,https://rustih.ru/ivan-krylov-martyshka-i-ochk...,Мартышка к старости слаба глазами стала;\nА у ...
2,Иван Крылов — Волк и Волчонок: Стих,https://rustih.ru/ivan-krylov-volk-i-volchonok/,"Волчонка Волк, начав помалу приучать\nОтцовски..."
3,Иван Крылов — Волк и журавль (Басня): Стих,https://rustih.ru/ivan-krylov-volk-i-zhuravl-b...,"Что волки жадны, всякий знает:\nВолк, евши, ни..."
4,Иван Крылов — Музыканты (Басня): Стих,https://rustih.ru/ivan-krylov-muzykanty-basnya/,Сосед соседа звал откушать;\nНо умысел другой ...


### Посмотрим пропуски в наших данных и удалим их

In [ ]:
df1 = data[data.isna().any(axis=1)]
df1.head()
data.dropna(inplace=True)

### Добавим колонки 'author' & 'title'

In [ ]:
authors = []
titles = []

for index, row in data.iterrows():

    book = row['Название'].split('—')
    if len(book) == 2:
        author = book[0]
        title = book[1]

        if ': Стих' in title:
            title = title.replace(': Стих', '')
        if ' (Басня)' in title:
            title = title.replace(' (Басня)', '')
        if title[0] == ' ':
          title = title[:0] + title[1:]
        if title[len(title) - 1] == ' ':
          title = title[:len(title) - 1] + title[len(title):]

        if author[0] == ' ':
          author = author[:0] + author[1:]
        if author[len(author) - 1] == ' ':
          author = author[:len(author) - 1] + author[len(author):]
    elif len(book) == 3:
      author = book[0]
      title = book[1] + book[2]

      if title[0] == ' ':
          title = title[:0] + title[1:]
      if title[len(title) - 1] == ' ':
        title = title[:len(title) - 1] + title[len(title):]

      if author[0] == ' ':
        author = author[:0] + author[1:]
      if author[len(author) - 1] == ' ':
        author = author[:len(author) - 1] + author[len(author):]
    else:
        author = 'Unknown'
        title = book[0]

    authors.append(author)
    titles.append(title)

data['author'] = authors
data['title'] = titles

# Посмотрим статистику по авторам
data['author'].value_counts()

Александр Пушкин         163
Михаил Лермонтов         134
Владимир Высоцкий        124
Николай Некрасов         123
Федор Тютчев             123
Владимир Маяковский      123
Сергей Есенин            122
Анна Ахматова            117
Борис Пастернак          115
Александр Блок           113
Афанасий Фет             111
Иван Бунин               107
Марина Цветаева          107
Иосиф Бродский           106
Самуил Маршак            106
Эдуард Асадов            105
Роберт Рождественский    104
Николай Гумилев          104
Евгений Евтушенко        103
Иван Крылов              102
Сергей Михалков          102
Константин Бальмонт      101
Юлия Друнина             101
Агния Барто              101
Арсений Тарковский       100
Шекспир                  100
Омар Хайям               100
Вера Полозкова            99
Валерий Брюсов            99
Гёте                      99
Дмитрий Быков             97
Александр Твардовский     80
Корней Чуковский          42
Николай Гумилёв            1
Name: author, 

In [ ]:
# Удалим запись с Николаем Гумилевым, т.к. дф содержит всего 1 стих
data = data.drop(data[data['author'] == 'Николай Гумилёв'].index)

### Непосредственно предобработка

*   Загружаем стоп-слова для лемматизации слов из библиотеки NLTK.
*   Приведем текст к нижнему регистру.
*   Удалим стоп-слова из текста.
*   Приведем каждое слово к нормальной форме (лемматизация).
*   Удалим знаки припенания.
*   Запишем изменения в новый столбец 'text'.

In [ ]:
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('russian'))

for index, row in data.iterrows():
    text = row['Текст']

    # Приведем текст к нижнему регистру
    text = text.lower()

    # Удалим пунктуацию
    text = re.sub(r'[^\w\s]', '', text)

    # Разделим текст на слова
    words = nltk.word_tokenize(text)

    # Удалим стоп-слова
    words = [word for word in words if word not in stop_words]

    # Приведем слова к нормальной форме
    words = [lemmatizer.lemmatize(word) for word in words]

    # Соединим слова обратно в строку
    processed_text = ' '.join(words)

    data.at[index, 'text'] = processed_text

data['text'] = data['text'].str.replace('_', '')

### Добавим дополнительные признки

In [ ]:
# Длина в символах
data['number_in_characters'] = data['text'].apply(lambda x: len(x))

# Длина в словах
data['length_in_words'] = data['text'].apply(lambda x: len(x.split()))

# Количесвто уникальных слов
data['number_of_unique_words'] = data['text'].apply(lambda x: len(set(x.split())))

# Средняя длина слова
data['average_word_length'] = data['text'].apply(lambda x: sum(len(word) for word in x.split()) / len(x.split()))

In [ ]:
data = data.drop(data[data['number_in_characters'] <= 40].index)

### Удалим лишние столбцы

In [ ]:
data = data.drop(['Название', 'Ссылка', 'Текст'], axis=1)
data.head()

,author,title,text,number_in_characters,length_in_words,number_of_unique_words,average_word_length
0,Иван Крылов,Свинья под дубом,свинья дубом вековым наелась желудей досыта от...,458,67,62,5.835821
1,Иван Крылов,Мартышка и очки,мартышка старости слаба глазами стала людей сл...,451,66,60,5.833333
2,Иван Крылов,Волк и Волчонок,волчонка волк начав помалу приучать отцовским ...,736,106,94,5.952830
3,Иван Крылов,Волк и журавль,волки жадны всякий знает волк евши костей разб...,470,69,62,5.826087
4,Иван Крылов,Музыканты,сосед соседа звал откушать умысел хозяин музык...,345,48,46,6.187500


### Выгрузим данные

In [ ]:
data.to_csv("table_poems02.csv", index=False)
files.download("table_poems02.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Откоем файлик

In [ ]:
df = pd.read_csv('/content/table_poems02.csv', delimiter=',', quotechar='"', escapechar='\\')
df.head()

,author,title,text,number_in_characters,length_in_words,number_of_unique_words,average_word_length
0,Иван Крылов,Свинья под дубом,свинья дубом вековым наелась желудей досыта от...,458,67,62,5.835821
1,Иван Крылов,Мартышка и очки,мартышка старости слаба глазами стала людей сл...,451,66,60,5.833333
2,Иван Крылов,Волк и Волчонок,волчонка волк начав помалу приучать отцовским ...,736,106,94,5.952830
3,Иван Крылов,Волк и журавль,волки жадны всякий знает волк евши костей разб...,470,69,62,5.826087
4,Иван Крылов,Музыканты,сосед соседа звал откушать умысел хозяин музык...,345,48,46,6.187500


In [ ]:
df['author_id'] = pd.factorize(df['author'])[0]
idx = df.columns.get_loc('author_id')
df = df.reindex(columns=['author_id'] + list(df.columns[:idx]) + list(df.columns[idx+1:]))

In [ ]:
df['author_id'].value_counts()

17    163
1     134
26    124
11    123
25    122
18    122
2     122
12    117
22    115
7     113
27    111
10    107
3     107
4     106
15    106
19    105
21    104
5     103
23    103
0     102
13    102
14    101
24    101
6     101
29    100
30    100
31    100
16     99
8      99
28     97
32     96
20     80
9      42
Name: author_id, dtype: int64

### pymorphy2

Подсчитаем количество существительных, глаголов, прилагательных, наречий

In [ ]:
morph = pymorphy2.MorphAnalyzer()

# Функция для подсчета слов по частям речи
def count_pos_words(text):
    nouns = 0
    adjectives = 0
    verbs = 0
    adverbs = 0
    words = text.split()

    for word in words:
        # Приведем слово к нормальной форме
        normal_form = morph.parse(word)[0].normal_form
        pos = morph.parse(normal_form)[0].tag.POS
        if pos == 'NOUN':
            nouns += 1
        elif pos == 'ADJF' or pos == 'ADJS':
            adjectives += 1
        elif pos == 'VERB':
            verbs += 1
        elif pos == 'ADVB':
            adverbs += 1

    return nouns, adjectives, verbs, adverbs

df[['nouns', 'adjectives', 'verbs', 'adverbs']] = df['text'].apply(count_pos_words).apply(pd.Series)
df.head()

,author_id,author,title,text,number_in_characters,length_in_words,number_of_unique_words,average_word_length,nouns,adjectives,verbs,adverbs
0,0,Иван Крылов,Свинья под дубом,свинья дубом вековым наелась желудей досыта от...,458,67,62,5.835821,27,3,0,4
1,0,Иван Крылов,Мартышка и очки,мартышка старости слаба глазами стала людей сл...,451,66,60,5.833333,32,5,0,1
2,0,Иван Крылов,Волк и Волчонок,волчонка волк начав помалу приучать отцовским ...,736,106,94,5.952830,38,21,0,7
3,0,Иван Крылов,Волк и журавль,волки жадны всякий знает волк евши костей разб...,470,69,62,5.826087,25,11,0,3
4,0,Иван Крылов,Музыканты,сосед соседа звал откушать умысел хозяин музык...,345,48,46,6.187500,21,4,0,1


In [ ]:
df.to_csv("table_poems03.csv", index=False)
files.download("table_poems03.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## TF-IDF

In [ ]:
texts = df['text'].tolist()
labels = df['author_id'].tolist()

Разделим так, чтобы авторы из test были в train

In [ ]:
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

Преобразуем тексты стихотворений в векторные представления

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(texts_train)
X_test = vectorizer.transform(texts_test)

Используем SVC

In [ ]:
classifier = SVC()

In [ ]:
classifier.fit(X_train, labels_train)

SVC()

In [ ]:
accuracy = classifier.score(X_test, labels_test)
print(accuracy)

0.06940509915014165


### Word2Vec

In [ ]:
df = pd.read_csv('/content/table_poems03.csv', delimiter=',', quotechar='"', escapechar='\\')

Получим векторное представление стихотворений

In [ ]:
model = Word2Vec(sentences=df['text'], vector_size=100, window=5, min_count=1, workers=4)

def get_poem_vector(poem):
    poem_vector = [model.wv[word] for word in poem if word in model.wv]
    if poem_vector:
        return np.mean(poem_vector, axis=0)
    else:
        return np.zeros(model.vector_size)

df['text_vector'] = df['text'].apply(get_poem_vector)

In [ ]:
df.head()

,author_id,author,title,text,number_in_characters,length_in_words,number_of_unique_words,average_word_length,nouns,adjectives,verbs,adverbs,text_vector
0,0,Иван Крылов,Свинья под дубом,свинья дубом вековым наелась желудей досыта от...,458,67,62,5.835821,27,3,0,4,"[-0.08416013, 0.099379614, -0.012152412, -0.17..."
1,0,Иван Крылов,Мартышка и очки,мартышка старости слаба глазами стала людей сл...,451,66,60,5.833333,32,5,0,1,"[-0.07851018, 0.082571484, -0.0098892115, -0.1..."
2,0,Иван Крылов,Волк и Волчонок,волчонка волк начав помалу приучать отцовским ...,736,106,94,5.952830,38,21,0,7,"[-0.069486335, 0.07655293, 0.0071866694, -0.18..."
3,0,Иван Крылов,Волк и журавль,волки жадны всякий знает волк евши костей разб...,470,69,62,5.826087,25,11,0,3,"[-0.08798582, 0.08536402, 0.006266758, -0.1633..."
4,0,Иван Крылов,Музыканты,сосед соседа звал откушать умысел хозяин музык...,345,48,46,6.187500,21,4,0,1,"[-0.06377496, 0.09431085, 0.0025009643, -0.172..."


In [ ]:
data = df

authors = data['author_id'].unique()

def find_nearest_author(vector, data):
    # Рассчитаем косинусное расстояние между вектором и каждым вектором из датафрейма
    similarities = cosine_similarity(vector.reshape(1,-1), list(data['text_vector']))
    # Выберем индекс стихотворения с наибольшим сходством
    nearest_index = similarities.argmax()
    return data.iloc[nearest_index]['author_id']

data['predicted_author_id'] = data.apply(lambda row: find_nearest_author(row['text_vector'], data[data['author_id'] != row['author_id']]), axis=1)

In [ ]:
count = len(df[df['predicted_author_id'] == df['author_id']])

print(f'Количество строк с совпадением: {count}')

Количество строк с совпадением: 0


найс

### Logistic Regression

In [ ]:
X = np.vstack(df['text_vector'].to_numpy())
y = df['author_id'].to_numpy()

# Разделим так, чтобы авторы из test были в train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
model_log = LogisticRegression()
model_log.fit(X_train, y_train)

y_pred = model_log.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.0821529745042493
